In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('File/Vietnamese_hallucination_annotated.csv')
df.head()

,title,gemini_text,gemini_sentences,annotation,gemini_text_samples,annotation_note
0,Hà Nội,"Hà Nội, thủ đô của Việt Nam, nằm ở vị trí 21°0...","['Hà Nội, thủ đô của Việt Nam, nằm ở vị trí 21...","['accurate', 'minor_inaccurate', 'accurate', '...","['Hà Nội, thủ đô của Việt Nam, nằm ở tọa độ đị...","[None, 'Hà Nội chỉ nằm trên lưu vực Sông Hồng ..."
1,Thành phố Hồ Chí Minh,Thành phố Hồ Chí Minh (TP.HCM) nằm ở phía Đông...,['Thành phố Hồ Chí Minh (TP.HCM) nằm ở phía Đô...,"['accurate', 'accurate', 'minor_inaccurate', '...",['Thành phố Hồ Chí Minh (TP. HCM) nằm ở phía Đ...,"[None, None, 'Hồ Chí Minh nằm ở hạ lưu sông Sà..."
2,Vịnh Hạ Long,"Vịnh Hạ Long nằm ở phía đông bắc Việt Nam, thu...","['Vịnh Hạ Long nằm ở phía đông bắc Việt Nam, t...","['accurate', 'minor_inaccurate', 'accurate', '...","['Vịnh Hạ Long nằm ở phía đông bắc Việt Nam, t...","[None, '20°54′B 107°12′Đ', None, 'phía Tây và ..."
3,Sapa,"Sa Pa là một thị trấn thuộc tỉnh Lào Cai, Việt...","['Sa Pa là một thị trấn thuộc tỉnh Lào Cai, Vi...","['accurate', 'minor_inaccurate', 'minor_inaccu...","['Sa Pa là một thị trấn thuộc tỉnh Lào Cai, Vi...","[None, '22°20′8″B 103°50′31″Đ', '1.500 m – 1.8..."
4,Đà Nẵng,Đà Nẵng là một thành phố ven biển thuộc miền T...,['Đà Nẵng là một thành phố ven biển thuộc miền...,"['minor_inaccurate', 'minor_inaccurate', 'accu...",['Đà Nẵng là một thành phố trực thuộc trung ươ...,"['16°04′36″B 108°13′22″Đ', '1.284,88 km2, Quản..."


In [3]:
dc1 = pd.read_csv('File/double_check-1.csv')
dc1.head(2)

,title,sentence,old_annotation,new_annotation,note,check,de_xuat,chú giải,final
0,Chiến tranh Tống–Việt lần thứ nhất,Chiến tranh Tống - Việt lần thứ nhất (1075-107...,minor_inaccurate,major_inaccurate,Cả đoạn này hiểu nhầm sang Tống Việt lần thứ 2...,x,NaN,NaN,minor_inaccurate
1,Chiến tranh Tống–Việt lần thứ nhất,Chiến thắng này đã bảo vệ vững chắc chủ quyền ...,minor_inaccurate,major_inaccurate,Lãnh thổ của Đại Cồ Việt mới đúng và nhà này đ...,NaN,NaN,NaN,major_inaccurate


In [4]:
dc2 = pd.read_csv('File/double_check-2.csv')
dc2.head(2)

,title,gemini_sentences,annotation_before,annotation_after,note,check,chú thích,final
0,Hội An,Địa hình của Hội An chủ yếu là đồng bằng ven b...,minor_inaccurate,NaN,Này tìm ko ra thông tin nên cũng ko rõ,NaN,"này tui xem gg map, check 4 hướng ntn",minor_inaccurate
1,Đà Lạt,Đà Lạt nằm ở vị trí 11°55′B 108°28′Đ﻿ / ﻿11.91...,minor_inaccurate,accurate,Tùy lệch nhưng nó nằm trong.,ok,NaN,accurate


In [5]:
import ast
for col in df.columns[2:]:
    df[col] = df[col].apply(lambda x: ast.literal_eval(x))

In [6]:
for i in range(len(dc1)):
    row = dc1.iloc[i]
    title = row['title']
    sen = row['sentence']
    new_annotation = row['final']
    # print(title)
    # print(sen)
    # print(new_annotation)
    
    break

In [7]:
title_matches = df[df['title'] == 'Chiến tranh Tống–Việt lần thứ nhất']

title_matches.index
sen_idx = df.loc[23, 'gemini_sentences'].index('Chiến tranh Tống - Việt lần thứ nhất (1075-1077) là cuộc chiến tranh bảo vệ lãnh thổ của Đại Việt chống lại sự xâm lược của nhà Tống.')
sen_idx

0

In [8]:
# Process each row in dc1 to update annotations in main df
for i in range(len(dc1)):
    row = dc1.iloc[i]
    title = row['title']
    sen = row['sentence']
    new_annotation = row['final']
    
    # Find the matching title in main df
    title_matches = df[df['title'] == title]
    
    idx = title_matches.index[0]
        # Find the sentence index in the gemini_sentences list
    try:
        sen_idx = df.loc[idx, 'gemini_sentences'].index(sen)
        # Update the annotation at that index
        df.loc[idx, 'annotation'][sen_idx] = new_annotation
    except:
        print(title, sen, new_annotation)

In [9]:
for i in range(len(dc2)):
    row = dc2.iloc[i]
    title = row['title']
    sen = row['gemini_sentences']
    new_annotation = row['final']
    
    # Find the matching title in main df
    title_matches = df[df['title'] == title]
    
    idx = title_matches.index[0]
        # Find the sentence index in the gemini_sentences list
    try:
        sen_idx = df.loc[idx, 'gemini_sentences'].index(sen)
        # Update the annotation at that index
        df.loc[idx, 'annotation'][sen_idx] = new_annotation
    except:
        print(title, sen, new_annotation)

In [10]:
df.to_csv('File/Vietnamese_hallucination_annotated_checked.csv', index=False)

In [13]:
title = 'Mai Chí Thọ'
sen = 'Ông được biết đến với vai trò là Bí thư Trung ương Đảng, Chủ tịch Ủy ban Trung ương Mặt trận Tổ quốc Việt Nam, và là một trong những người đóng góp quan trọng vào việc xây dựng và phát triển đất nước.'
title_matches = df[df['title'] == title]
idx = title_matches.index[0]
sen_idx = df.loc[idx, 'gemini_sentences'].index(sen)
df.loc[idx, 'annotation'][sen_idx]

'minor_inaccurate'